<a href="https://colab.research.google.com/github/dash-ka/NLP_Entailment/blob/master/%D0%98%D1%81%D0%BF%D0%BE%D0%BB%D1%8C%D0%B7%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_Ollama_%D1%81_%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D1%8C%D1%8E_Google_Colab_%D0%B8_WSL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 шаг: устанавливаем WSL (Подсистема Windows для Linux)

Официальный гайд по установке: https://learn.microsoft.com/ru-ru/windows/wsl/install

Список возможных для установки систем:
`wsl --list --online`

Ставим нужную систему:
`wsl --install -d <Distribution Name>`, вместо `<Distribution Name>` пишем название нужной системы из списка выше

После этого поставится нужная система, остаётся только создать имя пользователя и придумать пароль (при вводе его не видно).
Если будут проблемки — вот пошаговая установка: https://youtu.be/NlObaul_XT4&t=324

# 2 шаг: устанавливаем Ollama на наш Linux

In [ ]:
curl https://ollama.ai/install.sh | sh

# 3 шаг: устанавливаем Ollama на Colab и запускаем сервер на Colab

In [1]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  7983    0  7983    0     0  26080      0 --:--:-- --:--:-- --:--:-- 26173
>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 0.0.0.0:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
# создаём сервер для использование модели Ollama
!pip install aiohttp pyngrok

import os
import asyncio

os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})  # обновляем переменную окружения LD_LIBRARY_PATH

async def run_process(cmd):  # функция принимает список команд cmd
  print('>>> starting', *cmd)  # и создаёт процесс через .create_subprocess_exec(), в котором выполняются команды
  p = await asyncio.subprocess.create_subprocess_exec(
      *cmd,
      stdout=asyncio.subprocess.PIPE,  # с помощью stdout перехватываем вывод из стандартного потока вывода процессора
      stderr=asyncio.subprocess.PIPE,  # также перехватываем поток ошибок
  )

  async def pipe(lines):
    async for line in lines:
      print(line.strip().decode('utf-8'))

  await asyncio.gather(  # ожидаем завершения вывода из потоков, используя asyncio.gather()
      pipe(p.stdout),
      pipe(p.stderr),
  )

await asyncio.gather(
    run_process(['ollama', 'serve']),
    run_process(['ngrok', 'http', '--log', 'stderr', '11434']),
)

>>> starting ollama serve
>>> starting ngrok http --log stderr 11434
Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is:

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIJyFved41kgBouVJ7Q4TgzzETEr5fNhNPJsSKVBgPvgD

2023/12/09 15:03:44 images.go:734: total blobs: 0
2023/12/09 15:03:44 images.go:741: total unused blobs removed: 0
2023/12/09 15:03:44 routes.go:787: Listening on 127.0.0.1:11434 (version 0.1.13)
t=2023-12-09T15:03:44+0000 lvl=info msg="no configuration paths supplied"
t=2023-12-09T15:03:44+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml
t=2023-12-09T15:03:44+0000 lvl=info msg="open config file" path=/root/.config/ngrok/ngrok.yml err=nil
t=2023-12-09T15:03:45+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040 allow_hosts=[]
t=2023-12-09T15:03:45+0000 lvl=warn msg="Your agent is deprecated. Please update to 999.0.0 or later at your earliest convenience." obj=tunnels.session min_

# 4 шаг: подключаемся к серверу на нашем Linux и запускаем Ollama

In [ ]:
export OLLAMA_HOST=https://21b1-34-42-23-222.ngrok.io  # сюда вписываем url из Colab

In [ ]:
ollama run mistral